# Filtering 2 Practice - Groupby

In this practice exercise, we will analyze a yelp dataset. Import pandas and numpy below and read in the data set. Then get rid of all rows with NaN.

In [1]:
import pandas as pd
import numpy as np
import math


df_yelp = pd.read_csv("Yelp_Data.csv")
df_yelp.shape

(72742, 40)

In [2]:
df_yelp.dropna(axis = 0, inplace=True) # axis=0 -> drop row

df_yelp.shape

(72739, 40)

## Aggregate Practice

**Question 1:** Find the average star rating for each state (You can ignore the number of reviews for each restaurant). Store the result df_1.

state
AL     3.500000
AR     5.000000
AZ     3.709510
BW     3.768531
CA     3.500000
EDH    3.807777
ELN    3.791667
FIF    3.700000
HAM    4.500000
IL     3.549728
KHL    3.500000
MA     5.000000
MLN    3.842466
MN     5.000000
NC     3.587095
NM     3.000000
NTH    2.000000
NV     3.682979
NW     4.500000
ON     3.577825
OR     2.500000
PA     3.699276
QC     3.579906
RP     4.083333
SC     3.568266
SCB    4.000000
TX     4.500000
WI     3.659335
Name: stars, dtype: float64

**Question 2:** For each state compute the avg number of stars taking into account the number of reviews they got. First create a column called "total_stars", which is stars*review.  Then use groupby to compute the avg of total_stars as well as a count for the number of restaurants rated in each state and store the result in df_2. After this groupby remove all states that have fewer than 100 reviews. Then use this df to create column that is the average star rating.

In [6]:
df_yelp["total_stars"] = df_yelp["stars"] * df_yelp["review_count"]

def f(group):
    return pd.Series({
        "totalll_stars": group["total_stars"].sum(), 
        "review_count": group["review_count"].sum()})

df_2 = df_yelp.groupby("state").apply(f)

df_2 = df_2.loc[df_2.review_count >= 100]

df_2["final_rating"] = df_2["totalll_stars"]/df_2["review_count"]

df_2


,totalll_stars,review_count,final_rating
state,,,
AZ,3415460.5,905891.0,3.770278
BW,31424.0,8408.0,3.737393
CA,427.5,126.0,3.392857
EDH,151256.0,39082.0,3.870222
IL,63702.5,17273.0,3.687981
MLN,5079.5,1283.0,3.959080
NC,531745.0,144071.0,3.690854
NV,3904524.0,1047481.0,3.727537
ON,18858.0,5191.0,3.632826


**Question 3**: Find the most popular zip code for restaurants rated in AZ and NV.  You will have to write a custom function called Get_Most_Popular_Zip.  Create the function below.

Now pick out the restaurants from the states of interest and perform the group by and store the result in df_zip.

In [8]:
def Get_Most_Popular_Zip(group):
    # print(group.index) # 返回這個group 所有的 row number
    
    zip_count = {}
    for row_num in group.index:
        zip = group.loc[row_num, "full_address"].split()[-1]

        if zip in zip_count:
            zip_count[zip] += 1
        else:
            zip_count[zip] = 1
    
    zip_lst = list(zip_count.keys())
    counts = list(zip_count.values())

    pop_zip = zip_lst[counts.index(max(counts))]
    
    # return pop_zip

    return pd.Series({"MostPopularZip": pop_zip})



# 只保留 AZ NV
df_yelp_2 = df_yelp.loc[df_yelp.state.isin(["AZ", "NV"])]

df_zip = df_yelp_2.groupby("state").apply(Get_Most_Popular_Zip)

df_zip

,MostPopularZip
state,
AZ,85251
NV,89109


## Tranform Practice

**Question 4:** First select only the restaurants from AZ, IL, NC, and NV. Add a column called "Fraction_Reviews", which gives the fraction of reviews from the given state.

In [14]:
#Get the states we care about
df_yelp_3 = df_yelp.loc[df_yelp["state"].isin(["AZ", "IL", "NC", "NV"])].copy()

#Use a tranform to add a column that gives the total reviews from each state
df_yelp_3["Total_Reviews_By_State"] = df_yelp_3.groupby(by = ["state"]).review_count.transform(lambda x: x.sum())
df_yelp_3["Fraction_Reviews"] = df_yelp_3["review_count"]/df_yelp_3["Total_Reviews_By_State"]
df_yelp_3.head()

,business_id,city,full_address,latitude,longitude,name,review_count,stars,state,type,...,Asian Fusion,Diners,Greek,Vegetarian,Number_of_Checkins,Number_of_Tips,Number_of_Tip_Likes,total_stars,Total_Reviews_By_State,Fraction_Reviews
738,xXxJ7b_WUy0LBLIEM7614w,Belmont,"196 YMCA Dr Belmont, NC 28012",35.263370,-81.019970,Stowe Family YMCA,5,4.0,NC,business,...,0,0,0,0,55,0,0,20.0,144071,0.000035
739,fvKQpza6x2daBFXaGSm4Fg,Belmont,"7016 Wilkinson Blvd Belmont, NC 28012",35.248484,-81.021369,Carolina Auto Repair,3,3.5,NC,business,...,0,0,0,0,0,0,0,10.5,144071,0.000021
740,xkrhaKM1YtiLEcorBI5zcQ,Belmont,"6928 W Wilkinson Blvd Belmont, NC 28012",35.248667,-81.021242,Demetrios Restaurant,5,3.5,NC,business,...,0,0,0,0,8,0,0,17.5,144071,0.000035
741,qm6yyQUe8Ln29NzDfWqxVA,Belmont,"23 N Main St Belmont, NC 28012",35.243132,-81.038059,Cherubs Craft and Coffee Shop,5,5.0,NC,business,...,0,0,0,0,5,0,0,25.0,144071,0.000035
742,EZrCQtZxiEo1kkAYt2EQqw,Belmont,"660 Park St Belmont, NC 28012",35.253329,-81.027326,Wendy's,7,2.5,NC,business,...,0,0,0,0,16,3,0,17.5,144071,0.000049


## Filter Practice

**Question 5: **  Create a dataframe (df_well_reviewed) which only keeps states that have at least 100 different restaurants reviews. 

In [20]:
df_yelp["num_res"] = df_yelp.groupby("state").business_id.transform('count')
df_yelp.loc[df_yelp.num_res >= 100]


def has_at_least_100(group):
    return len(group) >= 100

df_yelp.groupby("state").filter(has_at_least_100)



,business_id,city,full_address,latitude,longitude,name,review_count,stars,state,type,...,Vietnamese,Asian Fusion,Diners,Greek,Vegetarian,Number_of_Checkins,Number_of_Tips,Number_of_Tip_Likes,total_stars,num_res
0,5UmKMjUEUNdYWqANhGckJw,Dravosburg,"4734 Lebanon Church Rd Dravosburg, PA 15034",40.354327,-79.900706,Mr Hoagie,4,4.5,PA,business,...,0,0,0,0,0,0,0,0,18.0,3731
1,UsFtqoBl7naz8AVUBZMjQQ,Dravosburg,"202 McClure St Dravosburg, PA 15034",40.350553,-79.886814,Clancy's Pub,4,3.5,PA,business,...,0,0,0,0,0,0,0,0,14.0,3731
2,3eu6MEFlq2Dg7bQh8QbdOg,Dravosburg,"1 Ravine St Dravosburg, PA 15034",40.350956,-79.889059,Joe Cislo's Auto,3,5.0,PA,business,...,0,0,0,0,0,0,0,0,15.0,3731
3,cE27W9VPgO88Qxe4ol6y_g,Bethel Park,"1530 Hamilton Rd Bethel Park, PA 15234",40.354115,-80.014660,Cool Springs Golf Center,5,2.5,PA,business,...,0,0,0,0,0,9,1,0,12.5,3731
4,HZdLhv6COCleJMo7nPl-RA,Pittsburgh,"301 South Hills Village Pittsburgh, PA 15241",40.357620,-80.059980,Verizon,5,2.5,PA,business,...,0,0,0,0,0,0,0,0,12.5,3731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72737,g51EInNrq2qhD3wkK2Dv7g,Edinburgh,128 Pitt St Leith Edinburgh EH6 4DD,55.973480,-3.183825,Santa's Little Yelper Wrapping Party,4,5.0,EDH,business,...,0,0,0,0,0,0,1,0,20.0,3176
72738,11BQ6VK8UsaFa4KJPup6Hg,Phoenix,"3130 W Carefree Hwy Phoenix, AZ 85086",33.799752,-112.126752,Starbucks,8,3.0,AZ,business,...,0,0,0,0,0,29,2,0,24.0,32533
72739,g52y11QlSrJNNKy2xPvB2Q,Peoria,"20783 N 83rd Ave Ste 109 Peoria, AZ 85382",33.672747,-112.237907,Tip Toe Nails Spa,15,4.5,AZ,business,...,0,0,0,0,0,0,0,0,67.5,32533
72740,dgzU923a29Yb1M8_Rz6R4A,Pittsburgh,"2726 Penn Avenue Strip District Pittsburgh, PA...",40.456238,-79.976112,Bow Wow Doggie Daycare,6,3.5,PA,business,...,0,0,0,0,0,3,0,0,21.0,3731


**Question 6:** For states that have at least 1000 reviews over all restaurants, compute the total number of Diners. Store the result in a dataframe called df_diners.

In [24]:
df_yelp_at_least_1000 = df_yelp.groupby("state").filter(lambda x: x.review_count.sum() >=1000)

df_yelp_at_least_1000.groupby("state").Diners.sum()

state
AZ     95
BW      1
EDH     8
IL      8
MLN     2
NC     31
NV     48
ON      8
PA     51
QC      4
SC      2
TX      0
WI     16
Name: Diners, dtype: int64

**Question 7:** For each state that has at least two cities whose name starts with a P, compute the number of tips. For this you will need to write a custom function to compute the number of cities that start with a P.

In [14]:
def Count_P(group):
    


state
AZ     223405
BW        129
EDH      4395
NC      31845
NV     283645
PA      16602
QC       1240
Name: Number_of_Tips, dtype: int64